# Multiple Linear Regression with sklearn - Exercise Solution

You are given a real estate dataset. 

Real estate is one of those examples that every regression course goes through as it is extremely easy to understand and there is a (almost always) certain causal relationship to be found.

The data is located in the file: 'real_estate_price_size_year.csv'. 

You are expected to create a multiple linear regression (similar to the one in the lecture), using the new data. 

Apart from that, please:
-  Display the intercept and coefficient(s)
-  Find the R-squared and Adjusted R-squared
-  Compare the R-squared and the Adjusted R-squared
-  Compare the R-squared of this regression and the simple linear regression where only 'size' was used
-  Using the model make a prediction about an apartment with size 750 sq.ft. from 2009
-  Find the univariate (or multivariate if you wish - see the article) p-values of the two variables. What can you say about them?
-  Create a summary table with your findings

In this exercise, the dependent variable is 'price', while the independent variables are 'size' and 'year'.

Good luck!

## Import the relevant libraries

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
sns.set()

from sklearn.linear_model import LinearRegression
# and of course the actual regression (machine learning) module
from sklearn import linear_model


## Load the data

In [25]:
data = pd.read_csv('real_estate_price_size_year (1).csv')

In [26]:
data.describe()

,price,size,year
count,100.000000,100.000000,100.000000
mean,292289.470160,853.024200,2012.600000
std,77051.727525,297.941951,4.729021
min,154282.128000,479.750000,2006.000000
25%,234280.148000,643.330000,2009.000000
50%,280590.716000,696.405000,2015.000000
75%,335723.696000,1029.322500,2018.000000
max,500681.128000,1842.510000,2018.000000


## Create the regression

### Declare the dependent and the independent variables

In [29]:
x = data[['size','year']]
y = data['price']


### Regression

In [30]:
reg = LinearRegression()
reg.fit(x,y)

LinearRegression()

### Find the intercept

In [31]:
reg.intercept_

-5772267.017463279

### Find the coefficients

In [32]:
reg.coef_

array([ 227.70085401, 2916.78532684])

### Calculate the R-squared

In [33]:
reg.score(x,y)

0.7764803683276795

### Calculate the Adjusted R-squared

In [35]:
Radj = 1-(1-reg.score(x,y))*((x.shape[0]-1)/(x.shape[0]-x.shape[1]-1))

In [36]:
Radj

0.7718717161282502

### Compare the R-squared and the Adjusted R-squared

R-squared is a bit higher that means we are not penalized of using two variables


### Compare the Adjusted R-squared with the R-squared of the simple linear regression

Comparing the Adjusted R-squared with the R-squared of the simple linear regression (when only 'size' was used - a couple of lectures ago), we realize that 'Year' is not bringing too much value to the result.

### Making predictions

Find the predicted price of an apartment that has a size of 750 sq.ft. from 2009.

In [39]:
reg.predict([[750,2009]])

C:\Users\fxgibert\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([258330.34465995])

### Calculate the univariate p-values of the variables

In [40]:
from sklearn.feature_selection import f_regression

In [41]:
f_regression(x,y)

(array([285.92105192,   0.85525799]), array([8.12763222e-31, 3.57340758e-01]))

In [42]:
p_values = f_regression(x,y)[1]

In [43]:
p_values.round(3)

array([0.   , 0.357])

### Create a summary table with your findings

In [45]:
reg_summary = pd.DataFrame(data = x.columns.values, columns = ['Features'])
reg_summary ['Coefficients'] = reg.coef_
reg_summary ['P-values'] = p_values.round(3)
reg_summary

,Features,Coefficients,P-values
0,size,227.700854,0.000
1,year,2916.785327,0.357


We can observe that the year is not that relevant into computing the price

In [48]:
import scipy.stats as stat

class LinearRegression(linear_model.LinearRegression):
    """
    LinearRegression class after sklearn's, but calculate t-statistics
    and p-values for model coefficients (betas).
    Additional attributes available after .fit()
    are `t` and `p` which are of the shape (y.shape[1], X.shape[1])
    which is (n_features, n_coefs)
    This class sets the intercept to 0 by default, since usually we include it
    in X.
    """
    
    # nothing changes in __init__
    def __init__(self, fit_intercept=True, normalize=False, copy_X=True,
                 n_jobs=1, positive=False):
        self.fit_intercept = fit_intercept
        self.normalize = normalize
        self.copy_X = copy_X
        self.n_jobs = n_jobs
        self.positive = positive
    
    def fit(self, X, y, n_jobs=1):
        self = super(LinearRegression, self).fit(X, y, n_jobs)
        
        # Calculate SSE (sum of squared errors)
        # and SE (standard error)
        sse = np.sum((self.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
        se = np.array([np.sqrt(np.diagonal(sse * np.linalg.inv(np.dot(X.T, X))))])

        # compute the t-statistic for each feature
        self.t = self.coef_ / se
        # find the p-value for each feature
        self.p = np.squeeze(2 * (1 - stat.t.cdf(np.abs(self.t), y.shape[0] - X.shape[1])))
        return self

In [49]:
reg_with_pvalues = LinearRegression()
reg_with_pvalues.fit(x,y)

LinearRegression()

In [50]:
# The difference is that we can check what's contained in the local variable 'p' in an instance of the LinearRegression() class
reg_with_pvalues.p

array([0., 0.])

In [55]:
reg_with_pvalues.p.round(100)

array([0., 0.])

In [52]:
reg_summary ['P-values'] = reg_with_pvalues.p.round(3)

In [53]:
reg_summary

,Features,Coefficients,P-values
0,size,227.700854,0.0
1,year,2916.785327,0.0
